In [15]:
import pandas as pd
import numpy as np

from skrub import TableVectorizer
import xgboost as xgb

from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline

import holidays

from datetime import datetime

from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

from sklearn.compose import ColumnTransformer


from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

import optuna

import datetime

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error



In [16]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")

In [17]:
# Add jour ferie data
jour_feries = (
    pd.read_csv(
        "/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/jours_feries_metropole.csv",
        date_format="%Y%m%d%H"  # Ensure date format is handled correctly
    )
    .drop(columns=["annee", "zone"])  # Drop unnecessary columns
)

# Convert 'date' column to datetime
jour_feries['date'] = pd.to_datetime(jour_feries['date'])

# Filter rows based on the date range of df_train and df_test
jour_feries = jour_feries[
    (jour_feries["date"] >= df_train["date"].min() - datetime.timedelta(hours=1))
    & (jour_feries["date"] <= df_test["date"].max() + datetime.timedelta(hours=1))
]

In [18]:
# Add mouvements sociaux data :
mouvements_sociaux = (
    pd.read_csv(
        "/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/mouvements-sociaux-depuis-2002.csv",
        date_format="%Y%m%d%H",
        sep=";"
    )
    .drop(columns=['date_de_fin', 'Organisations syndicales', 'Métiers ciblés par le préavis',
                   'Population devant travailler ciblee par le préavis', 'Nombre de grévistes du préavis'])  # Drop unnecessary columns
)

mouvements_sociaux['Date'] = pd.to_datetime(mouvements_sociaux['Date'])

mouvements_sociaux = mouvements_sociaux[
    (mouvements_sociaux["Date"] >= df_train["date"].min() - datetime.timedelta(hours=1))
    & (mouvements_sociaux["Date"] <= df_test["date"].max() + datetime.timedelta(hours=1))
]

mouvements_sociaux = mouvements_sociaux[mouvements_sociaux['Date'] != pd.Timestamp('2021-03-08')]

In [19]:
# Extract the date feature on different time scales :

fr_holidays = holidays.France()

def _encode_dates(X):
    X = X.copy()  # Modify a copy of X

    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # Creation of a binary variable depicting if the day is a weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Add a feature to indicate if the day is a holiday in France
    X["is_holiday"] = X["date"].apply(lambda d: 1 if d in fr_holidays else 0)

    # Add a feature to indicate if it is a jour férié in France
    X["is_jour_ferie"] = X["date"].dt.date.isin(jour_feries["date"]).astype(int)

    # Add a feature to indicate if it is a jour of "mouvement social" in France
    X["is_jour_mouvement_social"] = X["date"].dt.date.isin(mouvements_sociaux["Date"]).astype(int)

    return X

df_train = _encode_dates(df_train)
df_test = _encode_dates(df_test)


In [20]:
import geopandas as gpd
from shapely.geometry import Point

# To add an "arrondissement" feature based on latitute ande longitude
def arrondissement(X, shapefile_path="/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/arrondissements.shp"):

    arrondissements = gpd.read_file(shapefile_path)

    # Create a GeoDataFrame for the input dataset
    X = X.copy()  # Work on a copy of the dataset
    X["geometry"] = X.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
    gdf = gpd.GeoDataFrame(X, geometry="geometry", crs=arrondissements.crs)

    # Perform a spatial join to match points to arrondissements
    merged = gpd.sjoin(gdf, arrondissements, how="left", predicate="within")

    # Extract the arrondissement code (e.g., "c_ar") and fill missing values with 21
    X["district"] = merged["c_ar"].fillna(21).astype(int)

    # Drop the geometry column (optional, if not needed further)
    X = X.drop(columns=["geometry"])

    return X

df_train = arrondissement(df_train)
df_test = arrondissement(df_test)

In [21]:
df_train = df_train.drop(columns=['date'])
df_test = df_test.drop(columns=['date'])

In [22]:
# Preprocessing test :

ordinal_cols = [
    "counter_installation_date"
]

onehot_cols = [
    "counter_name",
    "site_name",
]

scale_cols = [
    "latitude",
    "longitude",
    "year",
    "month",
    "day",
    "weekday",
    "is_weekend",
    "hour",
    "is_holiday",
    "is_jour_ferie",
    "is_jour_mouvement_social",
    "district",
]

scaler = StandardScaler()
onehot = OneHotEncoder(sparse_output=False)
ordinal = OrdinalEncoder()


# Create the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("scale", scaler, scale_cols),
        ("onehot", onehot, onehot_cols),
        ("ordinal", ordinal, ordinal_cols),
    ]
)

# Define the full pipeline
def create_pipeline(params):
    model = XGBRegressor(**params, random_state=42)
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    return pipeline

In [23]:
X_train = df_train.drop(columns=["bike_count", "log_bike_count"])
y_train = df_train["log_bike_count"]

X_test = df_test.copy()

In [24]:
# Split the subset into train and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [25]:
# Define the Optuna objective function
def objective(trial):
    # Suggest hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-8, 10.0),
    }

    # Create pipeline with suggested parameters
    pipeline = create_pipeline(param)

    # Train the pipeline
    pipeline.fit(X_train_split, y_train_split)

    # Predict on validation set
    y_pred = pipeline.predict(X_val_split)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_val_split, y_pred))
    return rmse

# Create an Optuna study and optimize
study = optuna.create_study(direction="minimize")  # Minimize RMSE
study.optimize(objective, n_trials=50, timeout=1200)  # Adjust n_trials and timeout as needed

# Get the best parameters and score
print("Best Parameters:", study.best_params)
print("Best RMSE:", study.best_value)

[I 2024-12-11 11:53:00,048] A new study created in memory with name: no-name-5f8d610c-27f2-47c8-b116-4510bb7a3c25
[I 2024-12-11 11:53:02,824] Trial 0 finished with value: 0.6072635568223468 and parameters: {'n_estimators': 244, 'learning_rate': 0.1532926972547179, 'max_depth': 3, 'subsample': 0.8457742780192852, 'colsample_bytree': 0.8589763600760367, 'reg_alpha': 3.7435767305266285, 'reg_lambda': 3.151204674110778, 'min_child_weight': 4.448350523063254}. Best is trial 0 with value: 0.6072635568223468.
[I 2024-12-11 11:53:07,188] Trial 1 finished with value: 0.37962714960970817 and parameters: {'n_estimators': 266, 'learning_rate': 0.18574513070475784, 'max_depth': 7, 'subsample': 0.7086381474004485, 'colsample_bytree': 0.7904705480863794, 'reg_alpha': 7.2101243560555, 'reg_lambda': 1.1614692329011462, 'min_child_weight': 8.140298171687439}. Best is trial 1 with value: 0.37962714960970817.
[I 2024-12-11 11:53:12,720] Trial 2 finished with value: 0.3739702488747321 and parameters: {'n_e

Best Parameters: {'n_estimators': 456, 'learning_rate': 0.22651280285825093, 'max_depth': 10, 'subsample': 0.8718729809003489, 'colsample_bytree': 0.8094017347238014, 'reg_alpha': 7.596849031411172, 'reg_lambda': 6.261798376553035, 'min_child_weight': 7.383917899827265}
Best RMSE: 0.33695326936468617


In [26]:
# Train the final model with the best parameters on the full dataset
best_params = study.best_params
final_pipeline = create_pipeline(best_params)
final_pipeline.fit(X_train, y_train)


# best_model = xgb.XGBRegressor(**best_params, random_state=42)
# best_model.fit(X_train, y_train)  # Use the full training set for the final model

# Predict on the test set
# y_predictions = best_model.predict(X_test)
y_predictions = final_pipeline.predict(X_test)



In [27]:
print(y_predictions)

[0.33947682 1.6995281  2.1191804  ... 5.3035445  4.778556   3.5941298 ]


In [28]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions_XGBoost_Optuna.csv", index=False)